In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession # https://spark.apache.org/docs/1.6.1/sql-programming-guide.html
from pyspark.sql import functions as F

In [3]:
warehouse_location = os.path.abspath('spark-warehouse/test')
# Create the session
conf = SparkConf() \
    .set("spark.ui.port", "4041") \
    .set('spark.sql.warehouse.dir', warehouse_location)

# Create the context
sc = SparkContext(conf=conf)
spark = SparkSession.builder \
    .appName('Spark451') \
    .config("hive.metastore.uris", "thrift://localhost:9083") \
    .enableHiveSupport() \
    .getOrCreate()

sqlContext = SQLContext(sc)

In [11]:
%%time
# https://stackoverflow.com/questions/56927329/spark-option-inferschema-vs-header-true/56933052
df_master = (spark.read.format('csv')
    .option('header', 'true') # first row is column names.
    .option("inferSchema", "true") # set column types       
    .load('../data/irs990/master_file/*.csv'))

CPU times: user 1.89 ms, sys: 2.01 ms, total: 3.9 ms
Wall time: 3 s


In [13]:
df_master = df_master.toDF(*[c.lower() for c in df_master.columns]) # create lowercase columns
df_master.createOrReplaceTempView("master")

In [9]:
%%time
df_master.count()

CPU times: user 1.91 ms, sys: 1.83 ms, total: 3.74 ms
Wall time: 1.79 s


1757712

In [18]:
%%time
spark.sql('SELECT sum(revenue_amt) as sum from master').toPandas()

CPU times: user 7.92 ms, sys: 1.11 ms, total: 9.03 ms
Wall time: 1.2 s


,sum
0,2932332292337
